<a href="https://colab.research.google.com/github/c112154146-web/AICUP/blob/main/AI_CUP_2025_aortic_valve_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.232 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.1/112.6 GB disk)


In [ ]:
#下載資料集
import gdown
import os
import shutil

#下載training_image.zip
gdown.download("https://drive.google.com/file/d/1ZjW2PzUrNlx7z3sfSJuO45phUU-6XLgj/view?usp=drive_link","/content/training_image.zip", fuzzy=True)
#下載training_label.zip
gdown.download("https://drive.google.com/file/d/1tsqnsLFjciXNK37WYZz9H1ShJjhIf7DZ/view?usp=drive_link","/content/training_label.zip", fuzzy=True)
#下載訓練aortic_valve_colab.yaml
gdown.download("https://drive.google.com/file/d/1xy8VXNc3LNftygrLCuf4VFkWwMDZrqXT/view?usp=drive_link","/content/aortic_valve_colab.yaml", fuzzy=True)

In [ ]:
# 下載 yaml 後，執行這段來追加 test 路徑 (如果原本 yaml 裡面沒有寫的話)
import yaml

yaml_path = "/content/aortic_valve_colab.yaml"

# 讀取原本內容
with open(yaml_path, 'r') as f:
    data = yaml.safe_load(f)

# 更新路徑
data['train'] = './datasets/train/images'
data['val'] = './datasets/val/images'
data['test'] = './datasets/test/images'  # 加上這行

# 寫回檔案
with open(yaml_path, 'w') as f:
    yaml.dump(data, f)

print("YAML 檔已更新，包含 test 路徑")

In [ ]:
import os
import shutil

# --- 1. 基礎設定與解壓縮 (保持原本邏輯) ---
def find_patient_root(root):
    for dirpath, dirnames, filenames in os.walk(root):
        if any(d.startswith("patient") for d in dirnames):
            return dirpath
    return root

if not os.path.isdir("./training_image") and os.path.exists("training_image.zip"):
    os.makedirs("./training_image", exist_ok=True)
    !unzip -q training_image.zip -d ./training_image

if not os.path.isdir("./training_label") and os.path.exists("training_label.zip"):
    os.makedirs("./training_label", exist_ok=True)
    !unzip -q training_label.zip -d ./training_label

IMG_ROOT = find_patient_root("./training_image")
LBL_ROOT = find_patient_root("./training_label")

# --- 2. 建立並清空資料夾 ---
def ensure_clean_dir(path):
    if os.path.isdir(path):
        shutil.rmtree(path)
    os.makedirs(path, exist_ok=True)

# Train / Val (需要嚴格對應)
ensure_clean_dir("./datasets/train/images")
ensure_clean_dir("./datasets/train/labels")
ensure_clean_dir("./datasets/val/images")
ensure_clean_dir("./datasets/val/labels")

# Test (無條件搬移)
ensure_clean_dir("./datasets/test/images")
ensure_clean_dir("./datasets/test/labels")

# --- 3. 定義搬移函式 ---

def move_patients_strict(start, end, split):
    """
    [Train/Val 專用]
    嚴格模式：以 Label 為主。
    必須要有 .txt 標籤，才會去抓對應的圖片。確保訓練資料成對。
    """
    print(f"處理 {split} (嚴格模式): patient{start:04d}~{end:04d}")
    count = 0
    for i in range(start, end + 1):
        patient = f"patient{i:04d}"
        img_dir = os.path.join(IMG_ROOT, patient)
        lbl_dir = os.path.join(LBL_ROOT, patient)

        if not os.path.isdir(lbl_dir): continue

        for fname in os.listdir(lbl_dir):
            if not fname.endswith(".txt"): continue

            # 找到 Label，去抓對應 Image
            label_src = os.path.join(lbl_dir, fname)
            base, _ = os.path.splitext(fname)
            img_src = os.path.join(img_dir, base + ".png")

            if os.path.exists(img_src):
                shutil.copy2(img_src, f"./datasets/{split}/images/")
                shutil.copy2(label_src, f"./datasets/{split}/labels/")
                count += 1
    print(f"-> {split} 完成，共 {count} 張圖片。")

def move_patients_unconditional(start, end, split):
    """
    [Test 專用]
    無條件模式：以 Image 為主。
    只要看到圖片就搬過去，不管有沒有標籤 (有標籤就順便搬，沒有也沒差)。
    """
    print(f"處理 {split} (無條件模式): patient{start:04d}~{end:04d}")
    count = 0
    for i in range(start, end + 1):
        patient = f"patient{i:04d}"
        img_dir = os.path.join(IMG_ROOT, patient)
        lbl_dir = os.path.join(LBL_ROOT, patient) # 標籤資料夾 (選用)

        if not os.path.isdir(img_dir): continue

        # 遍歷圖片資料夾
        for fname in os.listdir(img_dir):
            if not fname.endswith(".png"): continue

            img_src = os.path.join(img_dir, fname)
            shutil.copy2(img_src, f"./datasets/{split}/images/")
            count += 1

            # 順便檢查有沒有標籤 (如果有就搬，方便之後算分數，沒有就算了)
            base, _ = os.path.splitext(fname)
            label_src = os.path.join(lbl_dir, base + ".txt")
            if os.path.exists(label_src):
                shutil.copy2(label_src, f"./datasets/{split}/labels/")

    print(f"-> {split} 完成，共 {count} 張圖片。")

# --- 4. 執行搬移 ---

# Train (1-40): 嚴格模式
move_patients_strict(1, 40, "train")

# Val (41-45): 嚴格模式
move_patients_strict(41, 45, "val")

# Test (46-50): 無條件模式 (圖片全部搬過去)
move_patients_unconditional(46, 50, "test")

print("所有資料處理完畢！")

In [ ]:
from ultralytics import YOLO

model = YOLO('yolo12n.pt') #初次訓練使用YOLO官方的預訓練模型，如要使用自己的模型訓練可以將'yolo12n.pt'替換掉
results = model.train(data="./aortic_valve_colab.yaml",
            epochs=30, #跑幾個epoch
            batch=16, #batch_size
            imgsz=640, #圖片大小640*640
            device=0 #使用GPU進行訓練
            )

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!zip -r '/content/datasets.zip' '/content/datasets'
from google.colab import files
files.download('/content/datasets.zip')

In [ ]:
!cp -r /content/runs/detect/train /content/drive/MyDrive/yolo_output/

In [ ]:
from ultralytics import YOLO

model = YOLO('/content/runs/detect/train/weights/best.pt') #將模型改成'best.pt'或其他訓練過的模型名稱
results = model.train(data="./aortic_valve_colab.yaml",
            epochs=10, #跑幾個epoch
            batch=16, #batch_size
            imgsz=640, #圖片大小640*640
            device=0 #使用GPU進行訓練
            )

In [ ]:
!zip -r '/content/train2.zip' '/content/runs/detect/train2' #打包訓練模型和結果
from google.colab import files
files.download('/content/train2.zip')